In [1]:
## 12 다중머신과 장치를 위한 분산 텐서플로 ##
import tensorflow as tf

In [2]:
!nvidia-smi

Tue Nov 24 20:04:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.67       Driver Version: 451.67       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106... WDDM  | 00000000:06:00.0  On |                  N/A |
|  0%   29C    P5    14W / 120W |    514MiB /  3072MiB |      9%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [4]:
## GPU 메모리 관리 ##
#1번 방법
config = tf.ConfigProto()
config.gpu_options.allow_growth = True #메모리 수요에 따라 할당

'''주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. 

Tensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'''

'주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. \n\nTensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'

In [ ]:
## GPU 메모리 관리 ##
#2번 방법
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3 #메모리 할당 비율 설정
#config.gpu_options.allow_growth = True #메모리 수요에 따라 할당, 두개 같이 사용 가능(추천하지는 않음)

In [ ]:
config.log_device_placement = True #배치자가 노드를 배치할 때마다 메세지 기록

In [4]:
tf.reset_default_graph()

with tf.device("/cpu:0"): #변수 a와 b를 cpu에 할당
    a = tf.Variable(3.0)
    b = tf.Variable(4.0)
    
c = a * b #c는 어떤 장치에도 할당되지 않았으므로 기본장치에 배치됨

with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(c))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:06:00.0, compute capability: 6.1

12.0


In [7]:
## 동적배치함수 ##
def variables_on_cpu(op): #연산이 할당될 장치이름 반환
    if op.type == "Variable":
        return "/cpu:0"
    else:
        return "/gpu:0"

tf.reset_default_graph()

with tf.device(variables_on_cpu):
    a = tf.Variable(3.0) #타입이 Variable이므로 cpu에 할당됨
    b = tf.Variable(4.0) #타입이 Variable이므로 cpu에 할당됨
    #b = tf.Variable(4) #gpu에는 int32형 변수 할당불가능
    c = a * b 
    
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(c))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:06:00.0, compute capability: 6.1

12.0


In [18]:
## 간접배치 ##
tf.reset_default_graph()

config.allow_soft_placement = True #처리할 커널이 없는 장치에 할당했을 때 cpu로 처리할수 있도록 함

In [19]:
tf.reset_default_graph()

with tf.device("/gpu:0"):
    b = tf.Variable(4) #cpu에 할당될 것임
    
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(b))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:06:00.0, compute capability: 6.1

4


In [22]:
## 제어의존성 ##
tf.reset_default_graph()

a = tf.Variable(1.0)
b = a + 2.0

with tf.control_dependencies([a, b]): #a,b가 평가된 두 x, y가 평가되게 함
    x = tf.constant(3.0)
    y = tf.constant(4.0)

z = x * y * b

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(z))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:06:00.0, compute capability: 6.1

36.0


In [6]:
## 12.2. 다중 머신의 다중 장치 ##
'''
클러스터 : 1개 이상의 태스크, 보통 여러 머신에 나뉘어 있음
태스크 : 하나의 잡에 속함, 하나의 잡은 이름이 부여된 태스크의 그룹(워커 / 파라미터 서버)
'''

'\n클러스터 : 1개 이상의 태스크, 보통 여러 머신에 나뉘어 있음\n태스크 : 하나의 잡에 속함, 하나의 잡은 이름이 부여된 태스크의 그룹(워커 / 파라미터 서버)\n'

<h2>로컬 서버</h2>

In [2]:
c = tf.constant("Hello distributed TensorFlow!")
server = tf.train.Server.create_local_server() #in-process 서버 형태로 단일 프로세스 클러스터를 생성
with tf.Session(server.target) as sess:
    print(sess.run(c))

b'Hello distributed TensorFlow!'


<h2>클러스터</h2>

In [3]:
#클러스터 명세

cluster_spec = tf.train.ClusterSpec({
    "ps":[
        "127.0.0.1:2221",  # /job:ps/task:0
        "127.0.0.1:2222",  # /job:ps/task:1
    ],
    "worker":[
        "127.0.0.1:2223",  # /job:worker/task:0
        "127.0.0.1:2224",  # /job:worker/task:1
        "127.0.0.1:2225",  # /job:worker/task:2
    ]})

# $ CUDA_VISIBLE_DEVICES="" python my_script.py # CPU만 사용하겠다는 뜻(파라미터 서버 설정)

In [4]:
task_ps0 = tf.train.Server(cluster_spec, job_name="ps", task_index=0) #첫번째 "ps" 태스크를 시작
task_ps1 = tf.train.Server(cluster_spec, job_name="ps", task_index=1) #두번째 "ps" 태스크를 시작
task_worker0 = tf.train.Server(cluster_spec, job_name="worker", task_index=0) #첫번째 "worker" 태스크를 시작
task_worker1 = tf.train.Server(cluster_spec, job_name="worker", task_index=1) #두번째 "worker" 태스크를 시작
task_worker2 = tf.train.Server(cluster_spec, job_name="worker", task_index=2) #세번째 "worker" 태스크를 시작

#server.join() #만약 서버를 실행하기만 하는 프로세스라면 다른서버가 종료될 때 까지 서버가 종료되지 않게 해야 함,
#실제로는 서버를 중지시키는 방법은 없기때문에 영원히 블록됨

<h2>장치와 서버간 파이프라이닝</h2>

In [12]:
tf.reset_default_graph()

with tf.device("/job:ps"): #ps잡의 1번째 태스크의 기본장치(cpu0)에 할당됨
    a = tf.Variable(1.0, name="a")

with tf.device("/job:worker/task:0"): #worker잡의 1번째 태스크의 기본장치(gpu0)에 할당됨
    b = a + 2
    
with tf.device("/job:worker/task:1/gpu:0"): #worker잡의 2번째 태스크의 1번째 gpu에 할당됨
    c = a + b
    
d = c * 2 #자신의 기본 장치에 할당됨

with tf.Session("grpc://127.0.0.1:2221") as sess:
    sess.run(a.initializer)
    print(d.eval())

8.0


In [30]:
tf.reset_default_graph()

with tf.device(tf.train.replica_device_setter( #라운드로빈 방식으로 변수들을 각 서버에 분산시킴
        ps_tasks=2, #ps태스크의 수, cluster=cluster_spec를 지정해서 ps잡에 있는 태스크의 수를 찾게 할수도 있음
        ps_device="/job:ps", #파마리터 서버 태스크 지정
        worker_device="/job:worker")): #워커 잡의 장치 지정
    v1 = tf.Variable(1.0, name="v1")  # pinned to /job:ps/task:0 (defaults to /cpu:0)
    v2 = tf.Variable(2.0, name="v2")  # pinned to /job:ps/task:1 (defaults to /cpu:0)
    v3 = tf.Variable(3.0, name="v3")  # pinned to /job:ps/task:0 (defaults to /cpu:0)
    s = v1 + v2            # pinned to /job:worker (defaults to task:0/cpu:0)
    with tf.device("/task:1"):
        p1 = 2 * s         # pinned to /job:worker/task:1 (defaults to /cpu:0)
        with tf.device("/cpu:0"):
            p2 = 3 * s     # pinned to /job:worker/task:1/cpu:0

config = tf.ConfigProto()
config.log_device_placement = True

with tf.Session("grpc://127.0.0.1:2221", config=config) as sess:
    sess.run([v1.initializer, v2.initializer, v3.initializer])

with tf.Session("grpc://127.0.0.1:2223", config=config) as sess:
    print(sess.run(p2))

9.0


<h2> 리소스 컨테이너를 사용해 여러 세션에서 상태 공유하기 </h2>

In [41]:
import sys

x = tf.Variable(0.0, name="x") #같은 클러스터에 있는 다른 세션에서 사용 가능
increment_x = tf.assign(x, x+1)

#init
with tf.Session("grpc://127.0.0.1:2223") as sess:
    sess.run(x.initializer)
    sess.run(increment_x)
    print(x.eval())

1.0


In [42]:
#다른 세션에서 같은 클러스터에 있는 변수 사용
with tf.Session("grpc://127.0.0.1:2224") as sess:
    sess.run(increment_x)
    print(x.eval())

with tf.Session("grpc://127.0.0.1:2225") as sess:
    sess.run(increment_x)
    print(x.eval())

2.0
3.0


In [17]:
#컨테이너 사용

tf.reset_default_graph()

#init
with tf.Session("grpc://127.0.0.1:2223") as sess:
    with tf.container("myProblem"):
        x = tf.Variable(10.0, name="x") #같은 클러스터에 있는 다른 세션에서 사용 가능
        increment_x = tf.assign(x, x+1)
    
    sess.run(x.initializer)
    sess.run(increment_x)
    print(x.eval())
    
tf.Session.reset("grpc://127.0.0.1:2223", ["myProblem"]) #myProblem 이름을 가진 컨테이너 리셋(컨테이너가 사용하는 모든 리소스 해제)
      
with tf.Session("grpc://127.0.0.1:2224") as sess:
    #sess.run(increment_x) #접근 불가능, 리소스  해제 됨
    pass

11.0
